Create a Kafka producer client using the given config, and produce 100 messages using `faker` of 
```
{
    'id': uuid4(),
    'name': name(),
    'email': email(),
    'timestamp': time.time()
}
```
to topic called `msds682.day3`

In [1]:
import json
import os
import time

from confluent_kafka import Producer
from dotenv import load_dotenv
from faker import Faker

In [2]:
load_dotenv()

True

In [3]:
conf = {
        'bootstrap.servers': os.environ['CONFLUENT_SERVER'],
        'security.protocol': 'SASL_SSL',
        'sasl.mechanism': 'PLAIN',
        'sasl.username': os.environ['CONFLUENT_API_KEY'],
        'sasl.password': os.environ['CONFLUENT_API_SECRET'],
    }

In [4]:
# Create the producer instance
producer = Producer(conf)

# Initialize data generator
fake = Faker()

In [ ]:
def delivery_report(err, msg):
    """
    Called once for each message produced to indicate the delivery status.
    """
    if err is not None:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message {msg.key()} delivered to topic '{msg.topic()}' to Partition {msg.partition()}")

%6|1761262945.077|GETSUBSCRIPTIONS|rdkafka#producer-1| [thrd:main]: Telemetry client instance id changed from AAAAAAAAAAAAAAAAAAAAAA to 2Cygn3oRTF6RTICnKVdMUw


In [6]:
for i in range(10):
    # Generate mock user data
    user_data = {
        'id': fake.uuid4(),
        'name': fake.name(),
        'email': fake.email(),
        'timestamp': time.time()
    }
    
    # Serialize the data to a JSON string
    json_data = json.dumps(user_data)
    
    # Produce the message to the Kafka topic
    # `on_delivery` specifies the callback function.
    producer.produce(
        topic='msds682.day3',
        key=str(user_data['id']),
        value=json_data,
        on_delivery=delivery_report
    )
    
    # Flush any outstanding or buffered messages to the Kafka broker.
    producer.poll(0)
    
    # Pause for a moment to simulate a real-time stream
    time.sleep(1)

producer.flush()

Message b'398d442b-1e15-4959-b0b9-c9c915d4740b' delivered to topic 'msds682.day3' to Partition 5
Message b'38e44973-178b-4ab7-a313-b8d829822cf3' delivered to topic 'msds682.day3' to Partition 0
Message b'3f88ccda-a954-4f85-8580-c344469f354f' delivered to topic 'msds682.day3' to Partition 4
Message b'f9b7891e-f12c-4f2d-98ca-fba23de48997' delivered to topic 'msds682.day3' to Partition 0
Message b'7cfd2166-1966-4621-b5b6-ab3c26cf08ac' delivered to topic 'msds682.day3' to Partition 0
Message b'126681d0-f466-4bfa-b65c-e403f5437c26' delivered to topic 'msds682.day3' to Partition 5
Message b'60df81d0-1713-462a-8e40-98cb562f7587' delivered to topic 'msds682.day3' to Partition 2
Message b'e2aae14f-6c4e-4fb9-aec1-275dca284840' delivered to topic 'msds682.day3' to Partition 5
Message b'8be60074-1f26-4b0f-a4e9-e138fc7995cb' delivered to topic 'msds682.day3' to Partition 4
Message b'14bfde56-7943-44ca-8d41-38bdf918f16a' delivered to topic 'msds682.day3' to Partition 3


0

Example - What happens without poll() and flush()?

In [7]:
while True:
    user_data = {
        'id': fake.uuid4(),
        'name': fake.name(),
        'email': fake.email(),
        'timestamp': time.time()
    }
    
    json_data = json.dumps(user_data)
    
    producer.produce(
        topic='msds682.day3',
        key=str(user_data['id']),
        value=json_data,
    )

KeyboardInterrupt: 